<a href="https://colab.research.google.com/github/MathieuForge/Moneyradar/blob/main/banques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Charger les CSV
bnp = pd.read_csv('bnp.csv')
ca = pd.read_csv('ca.csv')
soge = pd.read_csv('soge.csv')

# Renommer les colonnes 'Price' pour chaque DataFrame pour éviter les conflits
bnp = bnp.rename(columns={'Price': 'Price_BNP'})
ca = ca.rename(columns={'Price': 'Price_CA'})
soge = soge.rename(columns={'Price': 'Price_SOGE'})

# Fusionner les DataFrames sur la colonne 'Date'
banque = pd.merge(bnp[['Date', 'Price_BNP']], ca[['Date', 'Price_CA']], on='Date', how='inner')
banque = pd.merge(banque, soge[['Date', 'Price_SOGE']], on='Date', how='inner')

# Afficher le DataFrame résultat
print(banque)


           Date  Price_BNP  Price_CA  Price_SOGE
0    09/01/2024      62.72     14.30       21.92
1    08/01/2024      62.58     14.15       21.83
2    07/01/2024      63.46     14.03       23.98
3    06/01/2024      59.53     12.74       21.92
4    05/01/2024      67.70     14.93       27.39
..          ...        ...       ...         ...
232  05/01/2005      52.70     19.13       70.37
233  04/01/2005      49.09     18.17       67.77
234  03/01/2005      52.70     19.04       70.64
235  02/01/2005      52.85     20.35       70.33
236  01/01/2005      53.38     20.74       67.33

[237 rows x 4 columns]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.dates as mdates
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from PIL import Image, ImageOps, ImageDraw
from datetime import datetime

# Fonction pour générer un masque circulaire autour d'un logo
def circular_mask(image_path, size):
    img = Image.open(image_path).convert("RGBA")
    img = img.resize(size, Image.LANCZOS)

    mask = Image.new("L", img.size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0) + img.size, fill=255)

    output = ImageOps.fit(img, mask.size, centering=(0.5, 0.5))
    output.putalpha(mask)

    return output

def init():
    line_bnp.set_data([], [])
    line_ca.set_data([], [])
    line_soge.set_data([], [])
    ax1.set_xlim(x_interp.min(), x_interp.max())
    ax1.set_ylim(0, max(y_interp_bnp.max(), y_interp_ca.max(), y_interp_soge.max()) * 1.1)
    ax1.set_xlabel('Année', fontsize=20)
    ax1.set_ylabel('Prix des actions', fontsize=20, color='white')

    global date_counter
    date_counter = ax1.text(
        0.05, 0.95, '', transform=ax1.transAxes,
        fontsize=28, color='white', verticalalignment='top'
    )

    global logos, annotations
    logos = []
    annotations = []

    return line_bnp, line_ca, line_soge, date_counter

def update(frame):
    if frame < len(x_interp):
        line_bnp.set_data(x_interp[:frame], y_interp_bnp[:frame])
        line_ca.set_data(x_interp[:frame], y_interp_ca[:frame])
        line_soge.set_data(x_interp[:frame], y_interp_soge[:frame])

    if frame > 0:
        current_max_bnp = np.nanmax(y_interp_bnp[:frame]) * 1.1
        current_max_ca = np.nanmax(y_interp_ca[:frame]) * 1.1
        current_max_soge = np.nanmax(y_interp_soge[:frame]) * 1.1
        ax1.set_ylim(0, max(current_max_bnp, current_max_ca, current_max_soge))

        margin_years = 3
        end_date = x_interp.max() + np.timedelta64(margin_years * 365, 'D')
        ax1.set_xlim(x_interp.min(), end_date)

        for logo in logos:
            logo.remove()
        for ann in annotations:
            ann.remove()

        logos.clear()
        annotations.clear()

        # Annotations des valeurs
        annotations.append(
            ax1.annotate(
                f"{y_interp_bnp[frame]:.2f} €", (x_interp[frame], y_interp_bnp[frame]),
                textcoords="offset points", xytext=(35, 0), ha='left',
                color=colors['BNP'], fontsize=24
            )
        )
        annotations.append(
            ax1.annotate(
                f"{y_interp_ca[frame]:.2f} €", (x_interp[frame], y_interp_ca[frame]),
                textcoords="offset points", xytext=(35, 0), ha='left',
                color=colors['CA'], fontsize=24
            )
        )
        annotations.append(
            ax1.annotate(
                f"{y_interp_soge[frame]:.2f} €", (x_interp[frame], y_interp_soge[frame]),
                textcoords="offset points", xytext=(35, 0), ha='left',
                color=colors['Soge'], fontsize=24
            )
        )

        # Charger les logos des banques
        logo_bnp = circular_mask("logo_bnp.png", (200, 200))
        logo_ca = circular_mask("logo_ca.png", (200, 200))
        logo_soge = circular_mask("logo_soge.png", (200, 200))

        imagebox_bnp = OffsetImage(logo_bnp, zoom=0.25, resample=True)
        imagebox_ca = OffsetImage(logo_ca, zoom=0.25, resample=True)
        imagebox_soge = OffsetImage(logo_soge, zoom=0.25, resample=True)

        logo_bnp_ab = AnnotationBbox(imagebox_bnp, (x_interp[frame], y_interp_bnp[frame]), frameon=False)
        logo_ca_ab = AnnotationBbox(imagebox_ca, (x_interp[frame], y_interp_ca[frame]), frameon=False)
        logo_soge_ab = AnnotationBbox(imagebox_soge, (x_interp[frame], y_interp_soge[frame]), frameon=False)

        logos.extend([ax1.add_artist(logo_bnp_ab), ax1.add_artist(logo_ca_ab), ax1.add_artist(logo_soge_ab)])

        date_counter.set_text(f"{str(x_interp[frame])[:4]}")

    return line_bnp, line_ca, line_soge, date_counter

# Convertir la colonne 'Date' en format datetime
banque['Date'] = pd.to_datetime(banque['Date'], errors='coerce')

# Remplacer les valeurs infinies par NaN
banque.replace([np.inf, -np.inf], np.nan, inplace=True)

# Convertir les colonnes en float pour l'interpolation
banque['Price_BNP'] = pd.to_numeric(banque['Price_BNP'], errors='coerce')
banque['Price_CA'] = pd.to_numeric(banque['Price_CA'], errors='coerce')
banque['Price_Soge'] = pd.to_numeric(banque['Price_Soge'], errors='coerce')

# Interpolation pour combler les valeurs manquantes
banque['Price_BNP'] = banque['Price_BNP'].interpolate()
banque['Price_CA'] = banque['Price_CA'].interpolate()
banque['Price_Soge'] = banque['Price_Soge'].interpolate()

# Vérifier les NaN restants
banque.ffill(inplace=True)
banque.fillna(0, inplace=True)

# Trier les dates
banque = banque.sort_values(by='Date')

# Appliquer un lissage avec une fenêtre de 2 points
banque['Price_BNP'] = banque['Price_BNP'].rolling(window=2).mean()
banque['Price_CA'] = banque['Price_CA'].rolling(window=2).mean()
banque['Price_Soge'] = banque['Price_Soge'].rolling(window=2).mean()

# Supprimer les lignes contenant des NaN
banque = banque.dropna(subset=['Price_BNP', 'Price_CA', 'Price_Soge'])

# Préparer les données pour l'animation
x_data = banque['Date'].values.astype('datetime64[D]')
total_frames = 20 * 60
time_interp = np.linspace(0, len(x_data) - 1, total_frames)
x_interp = np.interp(time_interp, np.arange(len(x_data)), x_data.astype(np.int64)).astype('datetime64[D]')
y_interp_bnp = np.interp(time_interp, np.arange(len(banque)), banque['Price_BNP'])
y_interp_ca = np.interp(time_interp, np.arange(len(banque)), banque['Price_CA'])
y_interp_soge = np.interp(time_interp, np.arange(len(banque)), banque['Price_Soge'])

# Créer la figure et les axes
fig, ax1 = plt.subplots(figsize=(12, 21.33), dpi=150)
fig.patch.set_facecolor('#0D0C2D')
ax1.set_facecolor('#0D0C2D')

# Retirer les bordures supérieure et droite
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_linewidth(2)
ax1.spines['left'].set_linewidth(2)

# Configurer l'axe des x pour afficher les années tous les 5 ans
ax1.xaxis.set_major_locator(mdates.YearLocator(5))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Couleurs des lignes
colors = {'BNP': 'white', 'CA': '#0EE7A8', 'Soge': '#49C3FB'}

# Configurer les axes et leurs étiquettes
ax1.spines['left'].set_color('white')
ax1.spines['bottom'].set_color('white')
ax1.tick_params(axis='x', colors='white', labelsize=12, width=2)
ax1.tick_params(axis='y', colors='white', labelsize=12, width=2)
ax1.yaxis.label.set_color('white')
ax1.xaxis.label.set_color('white')

# Lignes de données
line_bnp, = ax1.plot([], [], lw=5, color=colors['BNP'], label='BNP')
line_ca, = ax1.plot([], [], lw=5, color=colors['CA'], label='Crédit Agricole')
line_soge, = ax1.plot([], [], lw=5, color=colors['Soge'], label='Société Générale')

# Créer l'animation
ani = FuncAnimation(fig, update, frames=total_frames, init_func=init, interval=50, blit=True)

# Ajouter la légende
leg = ax1.legend(loc='lower right', fontsize=16)
leg.get_frame().set_facecolor('none')
leg.get_frame().set_edgecolor('none')
for text in leg.get_texts():
    text.set_color('white')

# Sauvegarder l'animation (optionnel)
ani.save("banque_animation.mp4", fps=60, dpi=150)
